**TESTE DE ANALISTA DE DADOS - SX NEGOCIOS**

  Roberto Souza


---
<br>

  **Preparação**:  
      Como todo o desenvolvimento foi realizado no COLAB, e assim que reiniciado, o google limpa os arquivos que foram feitos uploads.
      *Assim é necessario colocar o arquivo Microdados no ./content/*

<br>
---

  
  **Ferramentas Utilizadas**:
* Google Colab
      Python, PySpark, SQL e Pandas (Para leitura, tratamento de dados, queries, gravação de arquivos);  
      Shutil e OS (Compactação de arquivos .zip)  

* Workbench  
      Criação da Modelagem  

* PowerBI  
      Criação das visualizações


---

  **Observações**:   
      os arquivos foram salvo em parquet e csv;
      um para melhor utilizar no python e outro para ser carregado no dataviz.

---      

> Inicio do codigo

In [1]:
# Calculando o tempo de execução
from datetime import datetime
inicio = datetime.now()

In [2]:
# Importando bibliotecas
import numpy as np
import pandas as pd
import shutil
import os

In [3]:
#Instalando o pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=2ab849882e3e5732739a9d16d62f8097d7aa42041405a82970901f942ea57fbd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
#importando pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, greatest, expr

In [5]:
#criando sessão do spark
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [6]:
# definindo diretorios
dir_raw = './raw/'
dir_proc = './proc/'

Preparação:    
  Como foi realizado em COLAB e assim que reiniciado o google limpa os arquivos. É necessario colocar o arquivo Microdados no ./content

In [7]:
# carregando o arquivo do upload e gravando uma copia em parquet
df_enem = spark.read.option('encoding', 'latin1').csv(f'./MICRODADOS_ENEM_2020.csv', sep=';', header=True)
df_enem.write.mode('overwrite').parquet(dir_raw + "enem/")

In [8]:
# carregando o arquivo parquet
df_enem_raw = spark.read.load(path= dir_raw + 'enem/*.parquet', format='parquet')

In [9]:
# amostra do dataset
df_enem_raw.show(5)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [10]:
# criando uma função para realização de queries
# PARAM = query e numero de linhas a serem retornadas
def fn_query(qry, snum):
    return spark.sql(qry).show(n = snum, truncate=False)

In [11]:
# Criando uma View temporaria
df_enem_raw.createOrReplaceTempView("VW_ENEM")

In [12]:
# Realizando um tratamento em SQL manipulando o dataset de acordo com o dicionario de dados
# e gravando na view criada
dataset_enem = spark.sql("""

    SELECT

    BIGINT(NU_INSCRICAO) AS NU_INSCRICAO,
    BIGINT(NU_ANO) AS NU_ANO,

    TP_FAIXA_ETARIA AS COD_FAIXA_ETARIA,
    CASE
      WHEN TP_FAIXA_ETARIA = 1 THEN 'Menor de 17 anos'
      WHEN TP_FAIXA_ETARIA = 2 THEN '17 anos'
      WHEN TP_FAIXA_ETARIA = 3 THEN '18 anos'
      WHEN TP_FAIXA_ETARIA = 4 THEN '19 anos'
      WHEN TP_FAIXA_ETARIA = 5 THEN '20 anos'
      WHEN TP_FAIXA_ETARIA = 6 THEN '21 anos'
      WHEN TP_FAIXA_ETARIA = 7 THEN '22 anos'
      WHEN TP_FAIXA_ETARIA = 8 THEN '23 anos'
      WHEN TP_FAIXA_ETARIA = 9 THEN '24 anos'
      WHEN TP_FAIXA_ETARIA = 10 THEN '25 anos'
      WHEN TP_FAIXA_ETARIA = 11 THEN 'Entre 26 e 30 anos'
      WHEN TP_FAIXA_ETARIA = 12 THEN 'Entre 31 e 35 anos'
      WHEN TP_FAIXA_ETARIA = 13 THEN 'Entre 36 e 40 anos'
      WHEN TP_FAIXA_ETARIA = 14 THEN 'Entre 41 e 45 anos'
      WHEN TP_FAIXA_ETARIA = 15 THEN 'Entre 46 e 50 anos'
      WHEN TP_FAIXA_ETARIA = 16 THEN 'Entre 51 e 55 anos'
      WHEN TP_FAIXA_ETARIA = 17 THEN 'Entre 56 e 60 anos'
      WHEN TP_FAIXA_ETARIA = 18 THEN 'Entre 61 e 65 anos'
      WHEN TP_FAIXA_ETARIA = 19 THEN 'Entre 66 e 70 anos'
      WHEN TP_FAIXA_ETARIA = 20 THEN 'Maior de 70 anos'
    END NOM_FAIXA_ETARIA,

    CASE
      WHEN TP_SEXO = 'F' THEN 1
      WHEN TP_SEXO = 'M' THEN 2
      ELSE -1
    END COD_TP_SEXO,
    TP_SEXO AS SIG_SEXO,
    CASE
      WHEN TP_SEXO = 'F' THEN 'Feminino'
      WHEN TP_SEXO = 'M' THEN 'Masculino'
      ELSE 'Não Preenchido'
    END NOM_SEXO,

    IFNULL(INT(TP_ESTADO_CIVIL), -1) AS COD_ESTADO_CIVIL,
    CASE
      WHEN TP_ESTADO_CIVIL = 0 THEN 'Não informado'
      WHEN TP_ESTADO_CIVIL = 1 THEN 'Solteiro(a)'
      WHEN TP_ESTADO_CIVIL = 2 THEN 'Casado(a)/Mora com companheiro(a)'
      WHEN TP_ESTADO_CIVIL = 3 THEN 'Divorciado(a)/Desquitado(a)/Separado(a)'
      WHEN TP_ESTADO_CIVIL = 4 THEN 'Viúvo(a)'
      END NOM_ESTADO_CIVIL,

    IFNULL(INT(TP_NACIONALIDADE), -1) AS COD_NACIONALIDADE,
    CASE
      WHEN TP_NACIONALIDADE = 0 THEN 'Não informado'
      WHEN TP_NACIONALIDADE = 1 THEN 'Brasileiro(a)'
      WHEN TP_NACIONALIDADE = 2 THEN 'Brasileiro(a) Naturalizado(a)'
      WHEN TP_NACIONALIDADE = 3 THEN 'Estrangeiro(a)'
      WHEN TP_NACIONALIDADE = 4 THEN 'Brasileiro(a) Nato(a), nascido(a) no exterior'
    END AS NOM_NACIONALIDADE,

    IFNULL(INT(TP_COR_RACA), -1) AS COD_RACA,
    CASE
      WHEN TP_COR_RACA = 0 THEN 'Não declarado'
      WHEN TP_COR_RACA = 1 THEN 'Branca'
      WHEN TP_COR_RACA = 2 THEN 'Preta'
      WHEN TP_COR_RACA = 3 THEN 'Parda'
      WHEN TP_COR_RACA = 4 THEN 'Amarela'
      WHEN TP_COR_RACA = 5 THEN 'Indígena'
    END NOM_COR_RACA,

    IFNULL(INT(TP_ESCOLA), -1) AS COD_ESCOLA,
    CASE
      WHEN TP_ESCOLA = 1 THEN 'Não Respondeu'
      WHEN TP_ESCOLA = 2 THEN 'Pública'
      WHEN TP_ESCOLA = 3 THEN 'Privada'
      WHEN TP_ESCOLA = 4 THEN 'Exterior'
    END NOM_ESCOLA,
    IFNULL(INT(TP_ENSINO), -1) AS COD_ENSINO,
    CASE
      WHEN TP_ENSINO = 1 THEN 'Ensino Regular'
      WHEN TP_ENSINO = 2 THEN 'Educação Especial - Modalidade Substitutiva'
      WHEN TP_ENSINO = 3 THEN 'Educação de Jovens e Adultos'
      ELSE 'Não Respondeu'
    END NOM_ENSINO,

    IFNULL(INT(TP_ST_CONCLUSAO), -1) AS COD_ST_CONCLUSAO,
    CASE
      WHEN TP_ST_CONCLUSAO = 1 THEN 'Já concluí o Ensino Médio'
      WHEN TP_ST_CONCLUSAO = 2 THEN 'Estou cursando e concluirei o Ensino Médio em 2020'
      WHEN TP_ST_CONCLUSAO = 3 THEN 'Estou cursando e concluirei o Ensino Médio após 2020'
      WHEN TP_ST_CONCLUSAO = 4 THEN 'Não concluí e não estou cursando o Ensino Médio'
      ELSE 'Não Respondeu'
    END NOM_ST_CONCLUSAO,

    IFNULL(INT(CO_MUNICIPIO_ESC), -1) AS COD_MUNICIPIO_ESC,
    IFNULL(NO_MUNICIPIO_ESC, 'Não Respondeu') AS NOM_MUNICIPIO_ESC,
    IFNULL(INT(CO_UF_ESC), -1) AS COD_UF_ESC,
    IFNULL(SG_UF_ESC, 'Não Respondeu') AS SIG_UF_ESC,
    IFNULL(INT(TP_LOCALIZACAO_ESC), -1) AS COD_LOCALIZACAO_ESC,
    CASE
        WHEN TP_LOCALIZACAO_ESC = 1 THEN 'Urbana'
        WHEN TP_LOCALIZACAO_ESC = 2 THEN 'Rural'
        ELSE 'Não Respondeu'
    END NOM_LOCALIZACAO_ESC,
    IFNULL(INT(TP_SIT_FUNC_ESC), -1) AS COD_SIT_FUNC_ESC,
    CASE
         WHEN TP_SIT_FUNC_ESC = 1 THEN 'Em atividade'
         WHEN TP_SIT_FUNC_ESC = 2 THEN 'Paralisada'
         WHEN TP_SIT_FUNC_ESC = 3 THEN 'Extinta'
         WHEN TP_SIT_FUNC_ESC = 4 THEN 'Escola extinta em anos anteriores'
        ELSE 'Não Respondeu'
    END NOM_SIT_FUNC_ESC,

    IFNULL(INT(TP_DEPENDENCIA_ADM_ESC), -1) AS COD_DEPENDENCIA_ADM_ESC,
     CASE
         WHEN TP_DEPENDENCIA_ADM_ESC = 1 THEN 'Federal'
         WHEN TP_DEPENDENCIA_ADM_ESC = 2 THEN 'Estadual'
         WHEN TP_DEPENDENCIA_ADM_ESC = 3 THEN 'Municipal'
         WHEN TP_DEPENDENCIA_ADM_ESC = 4 THEN 'Privada'
        ELSE 'Não Respondeu'
    END NOM_DEPENDENCIA_ADM_ESC,


    IFNULL(INT(IN_TREINEIRO), -1) AS COD_TREINEIRO,
    CASE
         WHEN IN_TREINEIRO = 1 THEN 'Sim'
         WHEN IN_TREINEIRO = 0 THEN 'Não'
    END NOM_TREINEIRO,

    IFNULL(INT(CO_MUNICIPIO_PROVA), -1) AS COD_MUNICIPIO_PROVA,
    IFNULL(NO_MUNICIPIO_PROVA, 'Não Respondeu') AS NOM_MUNICIPIO_PROVA,
    IFNULL(INT(CO_UF_PROVA), -1) AS COD_UF_PROVA,
    IFNULL(SG_UF_PROVA, 'Não Respondeu') AS SIG_UF_PROVA,


    FLOAT(NU_NOTA_REDACAO) NU_NOTA_REDACAO,
    FLOAT(NU_NOTA_CN) NU_NOTA_CN,
    FLOAT(NU_NOTA_CH) NU_NOTA_CH,
    FLOAT(NU_NOTA_LC) NU_NOTA_LC,
    FLOAT(NU_NOTA_MT) NU_NOTA_MT,

    INT(TP_PRESENCA_CN) AS COD_PRESENCA_CN,
    INT(TP_PRESENCA_CH) AS COD_PRESENCA_CH,
    INT(TP_PRESENCA_LC) AS COD_PRESENCA_LC,
    INT(TP_PRESENCA_MT) AS COD_PRESENCA_MT,

    IFNULL(Q001, -1 ) as Q001,
    IFNULL(Q002, -1 ) as Q002,
    IFNULL(Q003, -1 ) as Q003,
    IFNULL(Q004, -1 ) as Q004,
    IFNULL(Q005, -1 ) as Q005,
    IFNULL(Q006, -1 ) as Q006,
    IFNULL(Q007, -1 ) as Q007,
    IFNULL(Q008, -1 ) as Q008,
    IFNULL(Q009, -1 ) as Q009,
    IFNULL(Q010, -1 ) as Q010,
    IFNULL(Q011, -1 ) as Q011,
    IFNULL(Q012, -1 ) as Q012,
    IFNULL(Q013, -1 ) as Q013,
    IFNULL(Q014, -1 ) as Q014,
    IFNULL(Q015, -1 ) as Q015,
    IFNULL(Q016, -1 ) as Q016,
    IFNULL(Q017, -1 ) as Q017,
    IFNULL(Q018, -1 ) as Q018,
    IFNULL(Q019, -1 ) as Q019,
    IFNULL(Q020, -1 ) as Q020,
    IFNULL(Q021, -1 ) as Q021,
    IFNULL(Q022, -1 ) as Q022,
    IFNULL(Q023, -1 ) as Q023,
    IFNULL(Q024, -1 ) as Q024,
    IFNULL(Q025, -1 ) as Q025

     FROM VW_ENEM

""")


In [13]:
#dataset_enem.printSchema()

# gravando arquivo processado
dataset_enem.write.mode('overwrite').parquet(dir_proc + "enem/")

In [14]:
# carregando arquivo processado
dataset_enem_01 = spark.read.load(path = dir_proc + 'enem/*.parquet', format='parquet')
dataset_enem_01.limit(5).show()

+------------+------+----------------+------------------+-----------+--------+---------+----------------+--------------------+-----------------+--------------------+--------+------------+----------+-------------+----------+--------------+----------------+--------------------+-----------------+-------------------+----------+-------------+-------------------+-------------------+----------------+----------------+-----------------------+-----------------------+-------------+-------------+-------------------+-------------------+------------+------------+---------------+----------+----------+----------+----------+---------------+---------------+---------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|COD_FAIXA_ETARIA|  NOM_FAIXA_ETARIA|COD_TP_SEXO|SIG_SEXO| NOM_SEXO|COD_ESTADO_CIVIL|    NOM_ESTADO_CIVIL|COD_NACIONALIDADE|   NOM_NACIONALIDADE|COD_RACA|NOM_COR_RACA|COD_ESCO

In [15]:
# ver todos os nomes de columas para facilitar posteriormente
# e para verificar descomentar a linha abaixo

#dataset_enem_01.columns

Criando Dimensões e Fatos


In [16]:
# criar dimensões com campos unicos

# criando dimensões do participante
dim_prt_situacao_escola = dataset_enem_01.select ('COD_ST_CONCLUSAO','NOM_ST_CONCLUSAO').distinct()
dim_prt_faixa_etaria = dataset_enem_01.select ('COD_FAIXA_ETARIA','NOM_FAIXA_ETARIA').distinct()
dim_prt_sexo = dataset_enem_01.select ('COD_TP_SEXO','SIG_SEXO','NOM_SEXO').distinct()
dim_prt_estado_civil = dataset_enem_01.select ('COD_ESTADO_CIVIL','NOM_ESTADO_CIVIL').distinct()
dim_prt_nacionalidade = dataset_enem_01.select ('COD_NACIONALIDADE','NOM_NACIONALIDADE').distinct()
dim_prt_raca = dataset_enem_01.select ('COD_RACA','NOM_COR_RACA').distinct()

dim_prt_escola = dataset_enem_01.select ('COD_ESCOLA','NOM_ESCOLA').distinct()
dim_prt_ensino = dataset_enem_01.select ('COD_ENSINO','NOM_ENSINO').distinct()

# criando dimensões da prova
dim_prv_municipio = dataset_enem_01.select ('COD_MUNICIPIO_PROVA','NOM_MUNICIPIO_PROVA','COD_UF_PROVA','SIG_UF_PROVA').distinct()
dim_prv_uf = dataset_enem_01.select ('COD_UF_PROVA','SIG_UF_PROVA').distinct()

# criando dimensões da escola
dim_esc_dependencia = dataset_enem_01.select ('COD_DEPENDENCIA_ADM_ESC','NOM_DEPENDENCIA_ADM_ESC').distinct()
dim_esc_localizacao = dataset_enem_01.select ('COD_LOCALIZACAO_ESC','NOM_LOCALIZACAO_ESC').distinct()
dim_esc_situacao = dataset_enem_01.select ('COD_SIT_FUNC_ESC','NOM_SIT_FUNC_ESC').distinct()
dim_esc_uf = dataset_enem_01.select ('COD_UF_ESC','SIG_UF_ESC').distinct()

# criando e unindo UF
dim_uf = dim_esc_uf.union(dim_prv_uf).distinct()
dim_uf = dim_uf.withColumnRenamed("COD_UF_ESC","COD_UF").withColumnRenamed("SIG_UF_ESC","SIG_UF")

In [17]:
#dim_uf.orderBy('COD_UF').show(50)

In [18]:
# criando dimensões do dicionario para auxiliar e realizar tambem a modelagem

dim_presenca = spark.sql("""
    SELECT 0 AS COD_PRESENCA, 'Faltou à prova' AS NOM_PRESENCA UNION
    SELECT 1 AS COD_PRESENCA, 'Presente na prova' AS NOM_PRESENCA UNION
    SELECT 2 AS COD_PRESENCA, 'Eliminado na prova' AS NOM_PRESENCA
""")

dim_aux_r001e2 = spark.sql("""
    SELECT 'A' as COD_RESPOSTA, 'Nunca estudou.' NOM_RESPOSTA UNION
    SELECT 'B' as COD_RESPOSTA, 'Não completou a 4ª série/5º ano do Ensino Fundamental.' NOM_RESPOSTA UNION
    SELECT 'C' as COD_RESPOSTA, 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.' NOM_RESPOSTA UNION
    SELECT 'D' as COD_RESPOSTA, 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.' NOM_RESPOSTA UNION
    SELECT 'E' as COD_RESPOSTA, 'Completou o Ensino Médio, mas não completou a Faculdade.' NOM_RESPOSTA UNION
    SELECT 'F' as COD_RESPOSTA, 'Completou a Faculdade, mas não completou a Pós-graduação.' NOM_RESPOSTA UNION
    SELECT 'G' as COD_RESPOSTA, 'Completou a Pós-graduação.' NOM_RESPOSTA UNION
    SELECT 'H' as COD_RESPOSTA, 'Não sei.' NOM_RESPOSTA
""")

dim_aux_r003e4 = spark.sql("""
    SELECT 'A' AS COD_RESPOSTA, 'Grupo 1: Lavrador, agricultor sem empregados, bóia fria, criador de animais (gado, porcos, galinhas, ovelhas, cavalos etc.), apicultor, pescador, lenhador, seringueiro, extrativista.'NOM_RESPOSTA UNION
    SELECT 'B' AS COD_RESPOSTA, 'Grupo 2: Diarista, empregado doméstico, cuidador de idosos, babá, cozinheiro (em casas particulares), motorista particular, jardineiro, faxineiro de empresas e prédios, vigilante, porteiro, carteiro, office-boy, vendedor, caixa, atendente de loja, auxiliar administrativo, recepcionista, servente de pedreiro, repositor de mercadoria.'NOM_RESPOSTA UNION
    SELECT 'C' AS COD_RESPOSTA, 'Grupo 3: Padeiro, cozinheiro industrial ou em restaurantes, sapateiro, costureiro, joalheiro, torneiro mecânico, operador de máquinas, soldador, operário de fábrica, trabalhador da mineração, pedreiro, pintor, eletricista, encanador, motorista, caminhoneiro, taxista.'NOM_RESPOSTA UNION
    SELECT 'D' AS COD_RESPOSTA, 'Grupo 4: Professor (de ensino fundamental ou médio, idioma, música, artes etc.), técnico (de enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretor de imóveis, supervisor, gerente, mestre de obras, pastor, microempresário (proprietário de empresa com menos de 10 empregados), pequeno comerciante, pequeno proprietário de terras, trabalhador autônomo ou por conta própria.'NOM_RESPOSTA UNION
    SELECT 'E' AS COD_RESPOSTA, 'Grupo 5: Médico, engenheiro, dentista, psicólogo, economista, advogado, juiz, promotor, defensor, delegado, tenente, capitão, coronel, professor universitário, diretor em empresas públicas ou privadas, político, proprietário de empresas com mais de 10 empregados.'NOM_RESPOSTA UNION
    SELECT 'F' AS COD_RESPOSTA, 'Não sei.' NOM_RESPOSTA
""")

dim_aux_r006 = spark.sql("""
    SELECT 'A' AS COD_RESPOSTA,  'Nenhuma Renda' AS NOM_RESPOSTA UNION
    SELECT 'B' AS COD_RESPOSTA,  'Até R$ 1.045,00' AS NOM_RESPOSTA UNION
    SELECT 'C' AS COD_RESPOSTA,  'De R$ 1.045,01 até R$ 1.567,50' AS NOM_RESPOSTA UNION
    SELECT 'D' AS COD_RESPOSTA,  'De R$ 1.567,51 até R$ 2.090,00' AS NOM_RESPOSTA UNION
    SELECT 'E' AS COD_RESPOSTA,  'De R$ 2.090,01 até R$ 2.612,50' AS NOM_RESPOSTA UNION
    SELECT 'F' AS COD_RESPOSTA,  'De R$ 2.612,51 até R$ 3.135,00' AS NOM_RESPOSTA UNION
    SELECT 'G' AS COD_RESPOSTA,  'De R$ 3.135,01 até R$ 4.180,00' AS NOM_RESPOSTA UNION
    SELECT 'H' AS COD_RESPOSTA,  'De R$ 4.180,01 até R$ 5.225,00' AS NOM_RESPOSTA UNION
    SELECT 'I' AS COD_RESPOSTA,  'De R$ 5.225,01 até R$ 6.270,00' AS NOM_RESPOSTA UNION
    SELECT 'J' AS COD_RESPOSTA,  'De R$ 6.270,01 até R$ 7.315,00' AS NOM_RESPOSTA UNION
    SELECT 'K' AS COD_RESPOSTA,  'De R$ 7.315,01 até R$ 8.360,00' AS NOM_RESPOSTA UNION
    SELECT 'L' AS COD_RESPOSTA,  'De R$ 8.360,01 até R$ 9.405,00' AS NOM_RESPOSTA UNION
    SELECT 'M' AS COD_RESPOSTA,  'De R$ 9.405,01 até R$ 10.450,00' AS NOM_RESPOSTA UNION
    SELECT 'N' AS COD_RESPOSTA,  'De R$ 10.450,01 até R$ 12.540,00' AS NOM_RESPOSTA UNION
    SELECT 'O' AS COD_RESPOSTA,  'De R$ 12.540,01 até R$ 15.675,00' AS NOM_RESPOSTA UNION
    SELECT 'P' AS COD_RESPOSTA,  'De R$ 15.675,01 até R$ 20.900,00' AS NOM_RESPOSTA UNION
    SELECT 'Q' AS COD_RESPOSTA,  'Acima de R$ 20.900,00'  AS NOM_RESPOSTA
""")

dim_aux_r007 = spark.sql("""
    SELECT 'A' AS COD_RESPOSTA,  'Não.' AS NOM_RESPOSTA UNION
    SELECT 'B' AS COD_RESPOSTA,  'Sim, um ou dois dias por semana.' AS NOM_RESPOSTA UNION
    SELECT 'C' AS COD_RESPOSTA,  'Sim, três ou quatro dias por semana.' AS NOM_RESPOSTA UNION
    SELECT 'D' AS COD_RESPOSTA,  'Sim, pelo menos cinco dias por semana.'  AS NOM_RESPOSTA
""")

dim_aux_r008a24 = spark.sql("""
    SELECT 'A' AS COD_RESPOSTA,  'Não.' AS NOM_RESPOSTA UNION
    SELECT 'B' AS COD_RESPOSTA,  'Sim, um.' AS NOM_RESPOSTA UNION
    SELECT 'C' AS COD_RESPOSTA,  'Sim, dois.' AS NOM_RESPOSTA UNION
    SELECT 'D' AS COD_RESPOSTA,  'Sim, três.'  AS NOM_RESPOSTA UNION
    SELECT 'E' AS COD_RESPOSTA,  'Sim, quatro ou mais.'  AS NOM_RESPOSTA
""")

dim_aux_r018e20e21e23e25 = spark.sql("""
    SELECT 'A' AS COD_RESPOSTA,  'Não.' AS NOM_RESPOSTA UNION
    SELECT 'B' AS COD_RESPOSTA,  'Sim.' AS NOM_RESPOSTA
""")


In [19]:
dim_aux_perguntas = spark.sql("""
    SELECT 'Q001' AS COD_QUESTAO,	'Até que série seu pai, ou o homem responsável por você, estudou?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q002' AS COD_QUESTAO,	'Até que série sua mãe, ou a mulher responsável por você, estudou?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q003' AS COD_QUESTAO,	'A partir da apresentação de algumas ocupações divididas em grupos ordenados, indique o grupo que contempla a ocupação mais próxima da ocupação do seu pai ou do homem responsável por você. (Se ele não estiver trabalhando, escolha uma ocupação pensando no último trabalho dele).	' AS NOM_PERGUNTAS UNION
    SELECT 'Q004' AS COD_QUESTAO,	'A partir da apresentação de algumas ocupações divididas em grupos ordenados, indique o grupo que contempla a ocupação mais próxima da ocupação da sua mãe ou da mulher responsável por você. (Se ela não estiver trabalhando, escolha uma ocupação pensando no último trabalho dela).	' AS NOM_PERGUNTAS UNION
    SELECT 'Q005' AS COD_QUESTAO,	'Incluindo você, quantas pessoas moram atualmente em sua residência?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q006' AS COD_QUESTAO,	'Qual é a renda mensal de sua família? (Some a sua renda com a dos seus familiares.)	' AS NOM_PERGUNTAS UNION
    SELECT 'Q007' AS COD_QUESTAO,	'Em sua residência trabalha empregado(a) doméstico(a)?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q008' AS COD_QUESTAO,	'Na sua residência tem banheiro?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q009' AS COD_QUESTAO,	'Na sua residência tem quartos para dormir?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q010' AS COD_QUESTAO,	'Na sua residência tem carro?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q011' AS COD_QUESTAO,	'Na sua residência tem motocicleta?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q012' AS COD_QUESTAO,	'Na sua residência tem geladeira?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q013' AS COD_QUESTAO,	'Na sua residência tem freezer (independente ou segunda porta da geladeira)?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q014' AS COD_QUESTAO,	'Na sua residência tem máquina de lavar roupa? (o tanquinho NÃO deve ser considerado)' AS NOM_PERGUNTAS UNION
    SELECT 'Q015' AS COD_QUESTAO,	'Na sua residência tem máquina de secar roupa (independente ou em conjunto com a máquina de lavar roupa)?' AS NOM_PERGUNTAS UNION
    SELECT 'Q016' AS COD_QUESTAO,	'Na sua residência tem forno micro-ondas?' AS NOM_PERGUNTAS UNION
    SELECT 'Q017' AS COD_QUESTAO,	'Na sua residência tem máquina de lavar louça?' AS NOM_PERGUNTAS UNION
    SELECT 'Q018' AS COD_QUESTAO,	'Na sua residência tem aspirador de pó?' AS NOM_PERGUNTAS UNION
    SELECT 'Q019' AS COD_QUESTAO,	'Na sua residência tem televisão em cores?' AS NOM_PERGUNTAS UNION
    SELECT 'Q020' AS COD_QUESTAO,	'Na sua residência tem aparelho de DVD?' AS NOM_PERGUNTAS UNION
    SELECT 'Q021' AS COD_QUESTAO,	'Na sua residência tem TV por assinatura?' AS NOM_PERGUNTAS UNION
    SELECT 'Q022' AS COD_QUESTAO,	'Na sua residência tem telefone celular?' AS NOM_PERGUNTAS UNION
    SELECT 'Q023' AS COD_QUESTAO,	'Na sua residência tem telefone fixo?' AS NOM_PERGUNTAS UNION
    SELECT 'Q024' AS COD_QUESTAO,	'Na sua residência tem computador?	' AS NOM_PERGUNTAS UNION
    SELECT 'Q025' AS COD_QUESTAO,	'Na sua residência tem acesso à Internet?' AS NOM_PERGUNTAS
	""")


In [20]:
#gravando dimensões em parquet

#PARTICIPANTES
dim_prt_situacao_escola.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_prt_situacao_escola")
dim_prt_faixa_etaria.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_prt_faixa_etaria")
dim_prt_sexo.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_prt_sexo")
dim_prt_estado_civil.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_prt_estado_civil")
dim_prt_nacionalidade.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_prt_nacionalidade")
dim_prt_raca.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_prt_raca")

#PROVA
dim_prv_municipio.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_prv_municipio")

#ESCOLA ESC
dim_esc_dependencia.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_esc_dependencia")
dim_esc_localizacao.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_esc_localizacao")
dim_esc_situacao.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_esc_situacao")

#ESCOLA PART
dim_prt_escola.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_prt_escola")
dim_prt_ensino.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_prt_ensino")

#UF
dim_uf.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_uf")

In [21]:
#AUXILIARES
dim_presenca.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_presenca")
dim_aux_r001e2.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_aux_r001e2")
dim_aux_r003e4 .write.mode('overwrite').parquet(dir_proc + "dimensao/dim_aux_r003e4")
dim_aux_r006.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_aux_r006")
dim_aux_r007.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_aux_r007")
dim_aux_r008a24 .write.mode('overwrite').parquet(dir_proc + "dimensao/dim_aux_r008a24")
dim_aux_r018e20e21e23e25.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_aux_r018e20e21e23e25")
dim_aux_perguntas.write.mode('overwrite').parquet(dir_proc + "dimensao/dim_aux_pergunta")

In [22]:
# CRIANDO FATOS

In [23]:
# criando fato do participante
fat_participante = dataset_enem_01.select ('NU_INSCRICAO',
 'NU_ANO',
 'COD_FAIXA_ETARIA',
 'COD_TP_SEXO',
 'COD_ESTADO_CIVIL',
 'COD_NACIONALIDADE',
 'COD_RACA',
 'COD_ESCOLA',
 'COD_ENSINO',
 'COD_ST_CONCLUSAO',
 'COD_MUNICIPIO_ESC',
 'COD_UF_ESC',
 'COD_LOCALIZACAO_ESC',
 'COD_SIT_FUNC_ESC',
 'COD_DEPENDENCIA_ADM_ESC',
 'NOM_TREINEIRO',
 'COD_MUNICIPIO_PROVA',
 'NU_NOTA_REDACAO',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'COD_PRESENCA_CN',
 'COD_PRESENCA_CH',
 'COD_PRESENCA_LC',
 'COD_PRESENCA_MT')

In [24]:
fat_participante.show(3)

+------------+------+----------------+-----------+----------------+-----------------+--------+----------+----------+----------------+-----------------+----------+-------------------+----------------+-----------------------+-------------+-------------------+---------------+----------+----------+----------+----------+---------------+---------------+---------------+---------------+
|NU_INSCRICAO|NU_ANO|COD_FAIXA_ETARIA|COD_TP_SEXO|COD_ESTADO_CIVIL|COD_NACIONALIDADE|COD_RACA|COD_ESCOLA|COD_ENSINO|COD_ST_CONCLUSAO|COD_MUNICIPIO_ESC|COD_UF_ESC|COD_LOCALIZACAO_ESC|COD_SIT_FUNC_ESC|COD_DEPENDENCIA_ADM_ESC|NOM_TREINEIRO|COD_MUNICIPIO_PROVA|NU_NOTA_REDACAO|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|COD_PRESENCA_CN|COD_PRESENCA_CH|COD_PRESENCA_LC|COD_PRESENCA_MT|
+------------+------+----------------+-----------+----------------+-----------------+--------+----------+----------+----------------+-----------------+----------+-------------------+----------------+-----------------------+-------------

In [25]:
# criando fato do questionario
fat_questionario = dataset_enem_01.select (
    'NU_INSCRICAO',
    'Q001',    'Q002',    'Q003',    'Q004',    'Q005',    'Q006',    'Q007',    'Q008',    'Q009',
    'Q010',    'Q011',    'Q012',    'Q013',    'Q014',    'Q015',    'Q016',    'Q017',    'Q018',
    'Q019',    'Q020',    'Q021',    'Q022',    'Q023',    'Q024',    'Q025' )

In [26]:
fat_questionario.show(3)

+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|Q001|Q002|Q003|Q004|Q005|Q006|Q007|Q008|Q009|Q010|Q011|Q012|Q013|Q014|Q015|Q016|Q017|Q018|Q019|Q020|Q021|Q022|Q023|Q024|Q025|
+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|200006271946|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|
|200001195856|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|  -1|
|200001943954|   B|   C|   A|   D|   3|   B|   A|   B|   A|   A|   A|   B|   A|   B|   A|   B|   A|   A|   B|   A|   A|   A|   A|   A|   A|
+------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
only showing top 3 r

In [27]:
# gravando fatos em parquet

#PARTICIPANTE
fat_participante.write.mode('overwrite').parquet(dir_proc + "fato/fat_participante")

#PERGUNTAS
fat_questionario.write.mode('overwrite').parquet(dir_proc + "fato/fat_questionario")

In [28]:
# cagarregando arquivos parquet
# fatos e dimensoes

# PARTICIPANTE
df_dim_prt_situacao_escola = spark.read.load(path = dir_proc + "dimensao/dim_prt_situacao_escola/*.parquet", format='parquet')
df_dim_prt_faixa_etaria = spark.read.load(path = dir_proc + "dimensao/dim_prt_faixa_etaria/*.parquet", format='parquet')
df_dim_prt_sexo = spark.read.load(path = dir_proc + "dimensao/dim_prt_sexo/*.parquet", format='parquet')
df_dim_prt_estado_civil = spark.read.load(path = dir_proc + "dimensao/dim_prt_estado_civil/*.parquet", format='parquet')
df_dim_prt_nacionalidade = spark.read.load(path = dir_proc + "dimensao/dim_prt_nacionalidade/*.parquet", format='parquet')
df_dim_prt_raca = spark.read.load(path = dir_proc + "dimensao/dim_prt_raca/*.parquet", format='parquet')

#PROVA
df_dim_prv_municipio = spark.read.load(path = dir_proc + "dimensao/dim_prv_municipio/*.parquet", format='parquet')

#ESCOLA ESC
df_dim_esc_dependencia = spark.read.load(path = dir_proc + "dimensao/dim_esc_dependencia/*.parquet", format='parquet')
df_dim_esc_localizacao = spark.read.load(path = dir_proc + "dimensao/dim_esc_localizacao/*.parquet", format='parquet')
df_dim_esc_situacao = spark.read.load(path = dir_proc + "dimensao/dim_esc_situacao/*.parquet", format='parquet')

#ESCOLA PART
df_dim_prt_escola = spark.read.load(path = dir_proc + "dimensao/dim_prt_escola/*.parquet", format='parquet')
df_dim_prt_ensino = spark.read.load(path = dir_proc + "dimensao/dim_prt_ensino/*.parquet", format='parquet')

#UF
df_dim_uf = spark.read.load(path = dir_proc + "dimensao/dim_uf/*.parquet", format='parquet' )

#fato
fat_participante = spark.read.load(path= dir_proc + 'fato/fat_participante/*.parquet', format='parquet')
fat_questionario = spark.read.load(path= dir_proc + 'fato/fat_questionario/*.parquet', format='parquet')

#AUXILIARES
df_dim_presenca = spark.read.load(path = dir_proc + "dimensao/dim_presenca/*.parquet", format='parquet' )
df_dim_aux_r001e2  = spark.read.load(path=dir_proc + 'dimensao/dim_aux_r001e2/*.parquet', format='parquet')
df_dim_aux_r003e4  = spark.read.load(path=dir_proc + 'dimensao/dim_aux_r003e4/*.parquet', format='parquet')
df_dim_aux_r006  = spark.read.load(path=dir_proc + 'dimensao/dim_aux_r006/*.parquet', format='parquet')
df_dim_aux_r007  = spark.read.load(path=dir_proc + 'dimensao/dim_aux_r007/*.parquet', format='parquet')
df_dim_aux_r008a24  = spark.read.load(path=dir_proc + 'dimensao/dim_aux_r008a24/*.parquet', format='parquet')
df_dim_aux_r018e20e21e23e25  = spark.read.load(path=dir_proc + 'dimensao/dim_aux_r018e20e21e23e25/*.parquet', format='parquet')
df_dim_aux_perguntas  = spark.read.load(path=dir_proc + 'dimensao/dim_aux_pergunta/*.parquet', format='parquet')

In [29]:
#
# criando views para responder questionamentos do teste
#

#PARTICIPANTES
df_dim_prt_situacao_escola.createOrReplaceTempView("VW_DIM_PRT_SITUACAO")
df_dim_prt_faixa_etaria.createOrReplaceTempView("VW_DIM_PRT_FAIXA_ETARIA")
df_dim_prt_sexo.createOrReplaceTempView("VW_DIM_PRT_SEXO")
df_dim_prt_estado_civil.createOrReplaceTempView("VW_DIM_PRT_ESTADO_CIVIL")
df_dim_prt_nacionalidade.createOrReplaceTempView("VW_DIM_PRT_NACIONALIDADE")
df_dim_prt_raca.createOrReplaceTempView("VW_DIM_PRT_RACA")

#PROVA
df_dim_prv_municipio.createOrReplaceTempView("VW_DIM_MUNICIPIO")

#ESCOLA ESC
df_dim_esc_dependencia.createOrReplaceTempView("VW_DIM_ESC_DEPENDENCIA")
df_dim_esc_localizacao.createOrReplaceTempView("VW_DIM_ESC_LOCALIZACAO")
df_dim_esc_situacao.createOrReplaceTempView("VW_DIM_ESC_SITUACAO")

#ESCOLA PART
df_dim_prt_escola.createOrReplaceTempView("VW_DIM_PRT_ESCOLA")
df_dim_prt_ensino.createOrReplaceTempView("VW_DIM_PRT_ENSINO")

#UF
df_dim_uf.createOrReplaceTempView("VW_DIM_UF")

#PRESENÇA
df_dim_presenca.createOrReplaceTempView("VW_DIM_PRESENCA")

#fato
fat_participante.createOrReplaceTempView("VW_FAT_PARTICIPANTE")

In [30]:
# função a fim de localizar o nome do arquivo gerado em parquet
# ps: o parquet gera o arquivo com o nome proprio
#     com essa função resgatamos esse nome não o colocamos na mão

def localizaParquet(dir):
  path = "/content/proc/"+dir
  dir_list = os.listdir(path)

  for x in os.listdir(path):
    if x.endswith(".parquet"):
       name = path + x
  return name

In [31]:
# chamando a função recuperando os nomes dos parquet criados

name_dim_prt_situacao_escola = localizaParquet('dimensao/dim_prt_situacao_escola/')
name_dim_prt_faixa_etaria = localizaParquet('dimensao/dim_prt_faixa_etaria/')
name_dim_prt_sexo = localizaParquet('dimensao/dim_prt_sexo/')
name_dim_prt_estado_civil = localizaParquet('dimensao/dim_prt_estado_civil/')
name_dim_prt_nacionalidade = localizaParquet('dimensao/dim_prt_nacionalidade/')
name_dim_prt_raca = localizaParquet('dimensao/dim_prt_raca/')
name_dim_prv_municipio = localizaParquet('dimensao/dim_prv_municipio/')
name_dim_esc_dependencia = localizaParquet('dimensao/dim_esc_dependencia/')
name_dim_esc_localizacao = localizaParquet('dimensao/dim_esc_localizacao/')
name_dim_esc_situacao = localizaParquet('dimensao/dim_esc_situacao/')
name_dim_prt_escola = localizaParquet('dimensao/dim_prt_escola/')
name_dim_prt_ensino = localizaParquet('dimensao/dim_prt_ensino/')
name_dim_uf = localizaParquet('dimensao/dim_uf/')
name_dim_presenca = localizaParquet('dimensao/dim_presenca/')
name_fat_participante = localizaParquet('fato/fat_participante/')
name_fat_questionario = localizaParquet('fato/fat_questionario/')

name_dim_aux_r001e2  = localizaParquet('dimensao/dim_aux_r001e2/')
name_dim_aux_r003e4  = localizaParquet('dimensao/dim_aux_r003e4/')
name_dim_aux_r006  = localizaParquet('dimensao/dim_aux_r006/')
name_dim_aux_r007  = localizaParquet('dimensao/dim_aux_r007/')
name_dim_aux_r008a24  = localizaParquet('dimensao/dim_aux_r008a24/')
name_dim_aux_r018e20e21e23e25  = localizaParquet('dimensao/dim_aux_r018e20e21e23e25/')
name_dim_aux_perguntas  = localizaParquet('dimensao/dim_aux_pergunta/')

In [32]:
# trecho para gravar arquivos em csv a fim de colocar no dataviz

In [33]:
#
# lendo os arquivos parquet dessa vez em forma de dataset pandas
#

df_dim_prt_situacao_escola2 = pd.read_parquet(name_dim_prt_situacao_escola)
df_dim_prt_faixa_etaria2 = pd.read_parquet(name_dim_prt_faixa_etaria)
df_dim_prt_sexo2 = pd.read_parquet(name_dim_prt_sexo)
df_dim_prt_estado_civil2 = pd.read_parquet(name_dim_prt_estado_civil)
df_dim_prt_nacionalidade2 = pd.read_parquet(name_dim_prt_nacionalidade)
df_dim_prt_raca2 = pd.read_parquet(name_dim_prt_raca)

df_dim_prv_municipio2 = pd.read_parquet(name_dim_prv_municipio)

df_dim_esc_dependencia2 = pd.read_parquet(name_dim_esc_dependencia)
df_dim_esc_localizacao2 = pd.read_parquet(name_dim_esc_localizacao)
df_dim_esc_situacao2 = pd.read_parquet(name_dim_esc_situacao)

df_dim_prt_escola2 = pd.read_parquet(name_dim_prt_escola)
df_dim_prt_ensino2 = pd.read_parquet(name_dim_prt_ensino)

df_dim_uf2 = pd.read_parquet(name_dim_uf)

df_dim_presenca2 = pd.read_parquet(name_dim_presenca)

fat_participante2 = pd.read_parquet(name_fat_participante)
fat_questionario2 = pd.read_parquet(name_fat_questionario)

df_dim_aux_r001e2  = pd.read_parquet(name_dim_aux_r001e2)
df_dim_aux_r003e4  = pd.read_parquet(name_dim_aux_r003e4)
df_dim_aux_r006  = pd.read_parquet(name_dim_aux_r006)
df_dim_aux_r007  = pd.read_parquet(name_dim_aux_r007)
df_dim_aux_r008a24  = pd.read_parquet(name_dim_aux_r008a24)
df_dim_aux_r018e20e21e23e25  = pd.read_parquet(name_dim_aux_r018e20e21e23e25)
df_dim_aux_perguntas  = pd.read_parquet(name_dim_aux_perguntas)

In [34]:
#
# Gravando arquivos em CSV
#

# criando uma pasta csv
os.mkdir('./csv')

# gravando arquivos
df_dim_prt_situacao_escola2.to_csv('./csv/df_dim_prt_situacao_escola2.csv', sep=';', index=False,header=True)
df_dim_prt_faixa_etaria2.to_csv('./csv/df_dim_prt_faixa_etaria2.csv', sep=';', index=False,header=True)
df_dim_prt_sexo2.to_csv('./csv/df_dim_prt_sexo2.csv', sep=';', index=False,header=True)
df_dim_prt_estado_civil2.to_csv('./csv/df_dim_prt_estado_civil2.csv', sep=';', index=False,header=True)
df_dim_prt_nacionalidade2.to_csv('./csv/df_dim_prt_nacionalidade2.csv', sep=';', index=False,header=True)
df_dim_prt_raca2.to_csv('./csv/df_dim_prt_raca2.csv', sep=';', index=False,header=True)

df_dim_prv_municipio2.to_csv('./csv/df_dim_prv_municipio2.csv', sep=';', index=False,header=True)

df_dim_esc_dependencia2.to_csv('./csv/df_dim_esc_dependencia2.csv', sep=';', index=False,header=True)
df_dim_esc_localizacao2.to_csv('./csv/df_dim_esc_localizacao2.csv', sep=';', index=False,header=True)
df_dim_esc_situacao2.to_csv('./csv/df_dim_esc_situacao2.csv', sep=';', index=False,header=True)

df_dim_prt_escola2.to_csv('./csv/df_dim_prt_escola2.csv', sep=';', index=False,header=True)
df_dim_prt_ensino2.to_csv('./csv/df_dim_prt_ensino2.csv', sep=';', index=False,header=True)

df_dim_uf2.to_csv('./csv/df_dim_uf2.csv', sep=';', index=False,header=True)

fat_participante2.to_csv('./csv/fat_participante2.csv', sep=';', index=False,header=True)
fat_questionario2.to_csv('./csv/fat_questionario2.csv', sep=';', index=False,header=True)

In [35]:
df_dim_presenca2.to_csv('./csv/df_dim_presenca2.csv', sep=';', index=False,header=True)

df_dim_aux_r001e2.to_csv('./csv/df_dim_aux_r001e2.csv',sep=';', index=False, header=True)
df_dim_aux_r003e4 .to_csv('./csv/df_dim_aux_r003e4.csv',sep=';', index=False, header=True)
df_dim_aux_r006.to_csv('./csv/df_dim_aux_r006.csv',sep=';', index=False, header=True)
df_dim_aux_r007.to_csv('./csv/df_dim_aux_r007.csv',sep=';', index=False, header=True)
df_dim_aux_r008a24 .to_csv('./csv/df_dim_aux_r008a24.csv',sep=';', index=False, header=True)
df_dim_aux_r018e20e21e23e25.to_csv('./csv/df_dim_aux_r018e20e21e23e25.csv',sep=';', index=False, header=True)
df_dim_aux_perguntas.to_csv('./csv/df_dim_aux_perguntas.csv',sep=';', index=False, header=True)

In [36]:
#
# trecho para gravar todos os CSVs em um zip
# a fim de baixa-los posteriormente de uma vez.
#

shutil.make_archive('download_csv','zip','./csv')

'/content/download_csv.zip'

In [37]:
#
# descomentar abaixo para deletar a pasta CSV com todos os arquivos.
#

#!rm -rf csv
#!ls -lrt

**Trecho abaixo a fim de responder em sql/python os questionamentos feitos**

In [38]:
# ESCOLA COM MAIOR MEDIA

fn_query("""
  SELECT
  ESC.NOM_ESCOLA,
  ROUND(AVG( FAT.NU_NOTA_REDACAO + FAT.NU_NOTA_CN + FAT.NU_NOTA_CH + FAT.NU_NOTA_LC + FAT.NU_NOTA_MT ) / 5,2) MEDIA

  FROM VW_FAT_PARTICIPANTE FAT
    INNER JOIN VW_DIM_PRT_ESCOLA ESC
      ON FAT.COD_ESCOLA = ESC.COD_ESCOLA

    GROUP BY
        ESC.NOM_ESCOLA
    ORDER BY MEDIA DESC

""", 3)

+-------------+------+
|NOM_ESCOLA   |MEDIA |
+-------------+------+
|Privada      |629.48|
|Pública      |539.0 |
|Não Respondeu|538.19|
+-------------+------+



In [39]:
# ALUNO COM MAIOR MEDIA

fn_query("""
  SELECT
  FAT.NU_INSCRICAO  ,
  ROUND(AVG( FAT.NU_NOTA_REDACAO + FAT.NU_NOTA_CN + FAT.NU_NOTA_CH + FAT.NU_NOTA_LC + FAT.NU_NOTA_MT ) / 5,2) MEDIA

  FROM VW_FAT_PARTICIPANTE FAT

    GROUP BY
        NU_INSCRICAO
    ORDER BY MEDIA DESC

""", 3)

+------------+------+
|NU_INSCRICAO|MEDIA |
+------------+------+
|200004989284|836.14|
|200006014985|824.78|
|200004839616|821.86|
+------------+------+
only showing top 3 rows



In [40]:
#  % AUSENTES

fn_query("""
  SELECT
  COUNT(FAT.NU_INSCRICAO) AS TOT_INSCRITOS,
  COUNT(
    CASE
      WHEN FAT.COD_PRESENCA_CN = 0 THEN 1
    END) TOT_CIENCIAS_NAT,
  COUNT(
    CASE
      WHEN FAT.COD_PRESENCA_CH = 0 THEN 1
    END) TOT_CIENCIAS_HUM,
  COUNT(
    CASE
      WHEN FAT.COD_PRESENCA_LC = 0 THEN 1
    END) TOT_LINGUAGEM_COD,
  COUNT(
    CASE
      WHEN FAT.COD_PRESENCA_MT = 0 THEN 1
    END) TOT_MATEMATICA,

  COUNT(
    CASE
      WHEN 1=1
        AND FAT.COD_PRESENCA_CN = 0
        AND FAT.COD_PRESENCA_CH = 0
        AND FAT.COD_PRESENCA_LC = 0
        AND FAT.COD_PRESENCA_MT = 0
        THEN 1
    END) TOT_AUSENTES_TODAS_PROVAS,

    CONCAT(ROUND((TOT_AUSENTES_TODAS_PROVAS/TOT_INSCRITOS) * 100,2),'%') AS PERC_AUSENTES_TODAS_PROVAS

  FROM VW_FAT_PARTICIPANTE FAT

""",1)

+-------------+----------------+----------------+-----------------+--------------+-------------------------+--------------------------+
|TOT_INSCRITOS|TOT_CIENCIAS_NAT|TOT_CIENCIAS_HUM|TOT_LINGUAGEM_COD|TOT_MATEMATICA|TOT_AUSENTES_TODAS_PROVAS|PERC_AUSENTES_TODAS_PROVAS|
+-------------+----------------+----------------+-----------------+--------------+-------------------------+--------------------------+
|46740        |17137           |15869           |15869            |17137         |15806                    |33.82%                    |
+-------------+----------------+----------------+-----------------+--------------+-------------------------+--------------------------+



In [41]:
fn_query("""
  SELECT COUNT(NU_INSCRICAO) AS TOT_INSCRITOS FROM VW_FAT_PARTICIPANTE
  """,1)

+-------------+
|TOT_INSCRITOS|
+-------------+
|46740        |
+-------------+



In [42]:
#  MEDIA DISCIPLINA

fn_query("""
  SELECT
  COUNT(FAT.NU_INSCRICAO) AS TOT_INSCRITOS,
  ROUND(AVG( FAT.NU_NOTA_REDACAO + FAT.NU_NOTA_CN + FAT.NU_NOTA_CH + FAT.NU_NOTA_LC + FAT.NU_NOTA_MT ) / 5,2) MED_GERAL,
  ROUND(AVG( FAT.NU_NOTA_REDACAO),2) MED_REDACAO,
  ROUND(AVG( FAT.NU_NOTA_CN),2) MED_CIENCIAS_NAT,
  ROUND(AVG( FAT.NU_NOTA_CH),2) MED_CIENCIAS_HUM,
  ROUND(AVG( FAT.NU_NOTA_LC),2) MED_LINGUAGEM_COD,
  ROUND(AVG( FAT.NU_NOTA_MT),2) MED_MATEMATICA

  FROM VW_FAT_PARTICIPANTE FAT

""",1)

+-------------+---------+-----------+----------------+----------------+-----------------+--------------+
|TOT_INSCRITOS|MED_GERAL|MED_REDACAO|MED_CIENCIAS_NAT|MED_CIENCIAS_HUM|MED_LINGUAGEM_COD|MED_MATEMATICA|
+-------------+---------+-----------+----------------+----------------+-----------------+--------------+
|46740        |540.86   |629.3      |493.57          |521.86          |529.45           |519.57        |
+-------------+---------+-----------+----------------+----------------+-----------------+--------------+



In [43]:
#  MEDIA SEXO

fn_query("""
  SELECT
  COUNT(FAT.NU_INSCRICAO) AS TOT_INSCRITOS,
  SEX.NOM_SEXO,
  ROUND(AVG( FAT.NU_NOTA_REDACAO + FAT.NU_NOTA_CN + FAT.NU_NOTA_CH + FAT.NU_NOTA_LC + FAT.NU_NOTA_MT ) / 5,2) MED_GERAL,
  ROUND(AVG( FAT.NU_NOTA_REDACAO),2) MED_REDACAO,
  ROUND(AVG( FAT.NU_NOTA_CN),2) MED_CIENCIAS_NAT,
  ROUND(AVG( FAT.NU_NOTA_CH),2) MED_CIENCIAS_HUM,
  ROUND(AVG( FAT.NU_NOTA_LC),2) MED_LINGUAGEM_COD,
  ROUND(AVG( FAT.NU_NOTA_MT),2) MED_MATEMATICA

  FROM VW_FAT_PARTICIPANTE FAT
    INNER JOIN VW_DIM_PRT_SEXO SEX
      ON FAT.COD_TP_SEXO = SEX.COD_TP_SEXO
  GROUP BY SEX.NOM_SEXO

  ORDER BY MED_GERAL DESC

""",2)

+-------------+---------+---------+-----------+----------------+----------------+-----------------+--------------+
|TOT_INSCRITOS|NOM_SEXO |MED_GERAL|MED_REDACAO|MED_CIENCIAS_NAT|MED_CIENCIAS_HUM|MED_LINGUAGEM_COD|MED_MATEMATICA|
+-------------+---------+---------+-----------+----------------+----------------+-----------------+--------------+
|15234        |Masculino|549.55   |611.84     |509.52          |535.4           |530.96           |548.84        |
|31506        |Feminino |536.75   |637.58     |486.02          |515.44          |528.74           |505.73        |
+-------------+---------+---------+-----------+----------------+----------------+-----------------+--------------+



In [44]:
#  MEDIA ETNIA

fn_query("""
  SELECT
  COUNT(FAT.NU_INSCRICAO) AS TOT_INSCRITOS,
  RAC.NOM_COR_RACA,
  ROUND(AVG( FAT.NU_NOTA_REDACAO + FAT.NU_NOTA_CN + FAT.NU_NOTA_CH + FAT.NU_NOTA_LC + FAT.NU_NOTA_MT ) / 5,2) MED_GERAL,
  ROUND(AVG( FAT.NU_NOTA_REDACAO),2) MED_REDACAO,
  ROUND(AVG( FAT.NU_NOTA_CN),2) MED_CIENCIAS_NAT,
  ROUND(AVG( FAT.NU_NOTA_CH),2) MED_CIENCIAS_HUM,
  ROUND(AVG( FAT.NU_NOTA_LC),2) MED_LINGUAGEM_COD,
  ROUND(AVG( FAT.NU_NOTA_MT),2) MED_MATEMATICA

  FROM VW_FAT_PARTICIPANTE FAT
    INNER JOIN VW_DIM_PRT_RACA RAC
      ON FAT.COD_RACA = RAC.COD_RACA
  GROUP BY RAC.NOM_COR_RACA

  ORDER BY MED_GERAL DESC

""",8)

+-------------+-------------+---------+-----------+----------------+----------------+-----------------+--------------+
|TOT_INSCRITOS|NOM_COR_RACA |MED_GERAL|MED_REDACAO|MED_CIENCIAS_NAT|MED_CIENCIAS_HUM|MED_LINGUAGEM_COD|MED_MATEMATICA|
+-------------+-------------+---------+-----------+----------------+----------------+-----------------+--------------+
|15406        |Branca       |569.37   |668.78     |515.7           |547.63          |549.92           |554.74        |
|1054         |Amarela      |535.67   |626.99     |490.01          |515.58          |522.14           |515.39        |
|791          |Não declarado|532.91   |599.29     |490.96          |522.06          |523.8            |515.52        |
|22990        |Parda        |527.25   |613.01     |482.67          |508.62          |519.05           |502.83        |
|6233         |Preta        |519.15   |593.33     |477.29          |505.63          |517.26           |491.04        |
|266          |Indígena     |503.62   |553.26   

In [45]:
#
# trecho abaixo a fim de ver as tipagens de dados
#

In [46]:
df_dim_prt_situacao_escola.printSchema()
df_dim_prt_faixa_etaria.printSchema()
df_dim_prt_sexo.printSchema()
df_dim_prt_estado_civil.printSchema()
df_dim_prt_nacionalidade.printSchema()
df_dim_prt_raca.printSchema()

df_dim_prv_municipio.printSchema()

df_dim_esc_dependencia.printSchema()
df_dim_esc_localizacao.printSchema()
df_dim_esc_situacao.printSchema()

df_dim_prt_escola.printSchema()
df_dim_prt_ensino.printSchema()

df_dim_uf.printSchema()

root
 |-- COD_ST_CONCLUSAO: integer (nullable = true)
 |-- NOM_ST_CONCLUSAO: string (nullable = true)

root
 |-- COD_FAIXA_ETARIA: string (nullable = true)
 |-- NOM_FAIXA_ETARIA: string (nullable = true)

root
 |-- COD_TP_SEXO: integer (nullable = true)
 |-- SIG_SEXO: string (nullable = true)
 |-- NOM_SEXO: string (nullable = true)

root
 |-- COD_ESTADO_CIVIL: integer (nullable = true)
 |-- NOM_ESTADO_CIVIL: string (nullable = true)

root
 |-- COD_NACIONALIDADE: integer (nullable = true)
 |-- NOM_NACIONALIDADE: string (nullable = true)

root
 |-- COD_RACA: integer (nullable = true)
 |-- NOM_COR_RACA: string (nullable = true)

root
 |-- COD_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NOM_MUNICIPIO_PROVA: string (nullable = true)
 |-- COD_UF_PROVA: integer (nullable = true)
 |-- SIG_UF_PROVA: string (nullable = true)

root
 |-- COD_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- NOM_DEPENDENCIA_ADM_ESC: string (nullable = true)

root
 |-- COD_LOCALIZACAO_ESC: integer (nullable = 

In [47]:
fat_participante.printSchema()
fat_questionario.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: long (nullable = true)
 |-- COD_FAIXA_ETARIA: string (nullable = true)
 |-- COD_TP_SEXO: integer (nullable = true)
 |-- COD_ESTADO_CIVIL: integer (nullable = true)
 |-- COD_NACIONALIDADE: integer (nullable = true)
 |-- COD_RACA: integer (nullable = true)
 |-- COD_ESCOLA: integer (nullable = true)
 |-- COD_ENSINO: integer (nullable = true)
 |-- COD_ST_CONCLUSAO: integer (nullable = true)
 |-- COD_MUNICIPIO_ESC: integer (nullable = true)
 |-- COD_UF_ESC: integer (nullable = true)
 |-- COD_LOCALIZACAO_ESC: integer (nullable = true)
 |-- COD_SIT_FUNC_ESC: integer (nullable = true)
 |-- COD_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- NOM_TREINEIRO: string (nullable = true)
 |-- COD_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NU_NOTA_REDACAO: float (nullable = true)
 |-- NU_NOTA_CN: float (nullable = true)
 |-- NU_NOTA_CH: float (nullable = true)
 |-- NU_NOTA_LC: float (nullable = true)
 |-- NU_NOTA_MT: float (nullable =

In [48]:
df_dim_presenca.printSchema()

dim_aux_r001e2.printSchema()
dim_aux_r003e4.printSchema()
dim_aux_r006.printSchema()
dim_aux_r007.printSchema()
dim_aux_r008a24.printSchema()
dim_aux_r018e20e21e23e25.printSchema()
dim_aux_perguntas.printSchema()

root
 |-- COD_PRESENCA: integer (nullable = true)
 |-- NOM_PRESENCA: string (nullable = true)

root
 |-- COD_RESPOSTA: string (nullable = false)
 |-- NOM_RESPOSTA: string (nullable = false)

root
 |-- COD_RESPOSTA: string (nullable = false)
 |-- NOM_RESPOSTA: string (nullable = false)

root
 |-- COD_RESPOSTA: string (nullable = false)
 |-- NOM_RESPOSTA: string (nullable = false)

root
 |-- COD_RESPOSTA: string (nullable = false)
 |-- NOM_RESPOSTA: string (nullable = false)

root
 |-- COD_RESPOSTA: string (nullable = false)
 |-- NOM_RESPOSTA: string (nullable = false)

root
 |-- COD_RESPOSTA: string (nullable = false)
 |-- NOM_RESPOSTA: string (nullable = false)

root
 |-- COD_QUESTAO: string (nullable = false)
 |-- NOM_PERGUNTAS: string (nullable = false)



In [49]:
# tempo total de execução.

fim = datetime.now()
print(fim-inicio)

0:02:17.528731


> Final do Codigo